In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
def check_base_info(input_path, filename, output_path, minimal=False):
    df = pd.read_csv(input_path + filename + '.csv',  engine = 'python')
    col_names = df.columns.to_series()
    col_dtypes = df.dtypes
    col_na_count = df.isnull().sum()
    col_describe = df.describe().T
    col_unique = df.nunique()

    base_info = pd.concat([col_names, col_dtypes, col_na_count, col_unique], 
                          axis=1, keys=['col_names', 'col_dtypes', 
                                        'col_na_count', 'col_unique'], sort=False)
    base_info = pd.concat([base_info, col_describe], axis=1, sort=False)

    base_info.to_csv(output_path + filename + 'base_info.csv')
    
    df_report = ProfileReport(df, minimal=minimal)
    df_report.to_file(output_file = output_path + filename + '.html')

def check_missing(df):
    col_names = df.columns.to_series()
    col_dtypes = df.dtypes
    col_na_count = df.isnull().sum()
    col_describe = df.describe().T
    col_unique = df.nunique()

    base_info = pd.concat([col_names, col_dtypes, col_na_count, col_unique], 
                          axis=1, keys=['col_names', 'col_dtypes', 
                                        'col_na_count', 'col_unique'], sort=False)
    base_info = pd.concat([base_info, col_describe], axis=1, sort=False)
    
    return base_info

# 创建比赛ID，所有队伍交叉可能（不考虑区域），男队ID 为1000-1999, 数据中 ID数为1101-1467

In [3]:
input_path_mdata = f'./input/rawdata/MDataFiles_Stage1/'
input_path_mevents = f'./input/rawdata/'
output_path = f'./output/'

In [4]:
list_TeamID = [x for x in range(1101, 1468)]
list_Season = [x for x in range(1985, 2020)]

ID = []
Season = []
TeamID_a = []
TeamID_b = []

for item in  itertools.product(list_Season, list_TeamID, list_TeamID):
    if item[1] < item[2]:
        _ = str(item[0]) + '_' + str(item[1]) + '_' + str(item[2]) 
        ID.append(_)
        Season.append(item[0])
        TeamID_a.append(item[1])
        TeamID_b.append(item[2])

        
df_datamart = pd.concat([pd.Series(ID), pd.Series(Season), 
                pd.Series(TeamID_a), pd.Series(TeamID_b)], 
               axis=1, keys=['ID', 'Season', 'TeamID_a', 'TeamID_b' ])

In [5]:
check_missing(df_datamart)

,col_names,col_dtypes,col_na_count,col_unique,count,mean,std,min,25%,50%,75%,max
ID,ID,object,0,2350635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Season,Season,int64,0,35,2350635.0,2002.000000,10.099507,1985.0,1993.0,2002.0,2011.0,2019.0
TeamID_a,TeamID_a,int64,0,366,2350635.0,1222.666667,86.384173,1101.0,1150.0,1208.0,1284.0,1466.0
TeamID_b,TeamID_b,int64,0,366,2350635.0,1345.333333,86.384173,1102.0,1284.0,1360.0,1418.0,1467.0


In [6]:
print(df_datamart.head(100))

print(df_datamart.shape)

                ID  Season  TeamID_a  TeamID_b
0   1985_1101_1102    1985      1101      1102
1   1985_1101_1103    1985      1101      1103
2   1985_1101_1104    1985      1101      1104
3   1985_1101_1105    1985      1101      1105
4   1985_1101_1106    1985      1101      1106
5   1985_1101_1107    1985      1101      1107
6   1985_1101_1108    1985      1101      1108
7   1985_1101_1109    1985      1101      1109
8   1985_1101_1110    1985      1101      1110
9   1985_1101_1111    1985      1101      1111
10  1985_1101_1112    1985      1101      1112
11  1985_1101_1113    1985      1101      1113
12  1985_1101_1114    1985      1101      1114
13  1985_1101_1115    1985      1101      1115
14  1985_1101_1116    1985      1101      1116
15  1985_1101_1117    1985      1101      1117
16  1985_1101_1118    1985      1101      1118
17  1985_1101_1119    1985      1101      1119
18  1985_1101_1120    1985      1101      1120
19  1985_1101_1121    1985      1101      1121
20  1985_1101

# 加入target_p1，target_p2 此部分为对阵比赛结果，target_p1为分差，用score_diff表示，target_p2为胜负，用win表示
### [MRegularSeasonDetaileResults.csv] [MNCAATourneyDetailedResults.csv]两个文件中提取出胜负结果及分差，另加一列来区别常规赛和淘汰赛

In [7]:
df_regular_results = pd.read_csv(input_path_mdata + 'MRegularSeasonDetailedResults.csv', engine = 'python')
df_tourney_results = pd.read_csv(input_path_mdata + 'MNCAATourneyDetailedResults.csv', engine = 'python')

In [8]:
df_regular_results.head(5)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [9]:
df_tourney_results.head(5)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


### 说明：文件 [MRegularSeasonDetaileResults.csv] [MNCAATourneyDetailedResults.csv] 是文件 【MRegularSeasonCompactResults.csv】【MNCAATourneyCompactResults.csv】的扩展版本
### 前8列完全相同，后面追加的为整场比赛中的统计数据（此处要跟event区分开，event是发生每个活动都记录，每场比赛统计多次， 而detaile是每场比赛结束时，统计一次总数）
### WTO-turnovers committed- 赢队的失误次数 WBlk-blocks- 赢队的盖帽次数
### 每列说明： WFGM - 投篮投中次数（所有，包括2分，3分，不含罚篮） WFGA-投篮次数（比如5投3中 的5） WFGM3-3分球投中次数 WFGA3-3分球投篮次数 WFTM-罚球中次数 WFTA-罚球次数 
### WOR-进攻篮板 WDR-防守篮板 WAst-助攻 WTO-turnovers committed？？ WStl-抢断 WBlk-blocks？？ WPF-犯规

In [26]:
#Add a lable variable to the two datasets

df_regular_results['Label'] = 'Reg'
df_tourney_results['Label']= 'Tourney'

In [11]:
df_regular_results.shape

(87504, 35)

## Feature Engineering - Adding features for DataMart

### 1. Adding Score Difference

In [27]:
def score_diff(dfin):
    df = dfin.copy()
    df['id_diff'] = df['WTeamID'] - df['LTeamID'] #Variable used to create ID matched with datamart
    '''Rule: if Win team has smaller ID number than Lost team - > Score_diff = Wscore - LScore
            else: Score_diff = LScore - Wscore since the team ID order will be exchanged'''
    '''[Wloc] identifies the "location" of the winning team. 
            If the winning team was the home team, this value will be "H". 
            If the winning team was the visiting team, this value will be "A". 
            If it was played on a neutral court, then this value will be "N".'''

    df['win'] = df.apply(lambda x: 1 if x.id_diff<0 else 0, axis=1) 
    df['LLoc'] = df.apply(lambda x: 'H' if x.WLoc == 'A' else('A' if x.WLoc == 'H' else 'N'), axis =1)
    
    df['Team_a_Loc'] = df.apply(lambda x: x.WLoc if x.id_diff<0 else x.LLoc, axis=1)
    df['Team_a_FGM2_rate'] = df.apply(lambda x: (x.WFGM - x.WFGM3) / (x.WFGA - x.WFGA3) if x.id_diff <0 else (x.LFGM - x.LFGM3) / (x.LFGA - x.LFGA3), axis=1) 
    df['Team_a_FGM3_rate'] = df.apply(lambda x: x.WFGM3 / x.WFGA3 if x.id_diff <0 else x.LFGM3 / x.LFGA3, axis=1)
    df['Team_a_FTM_count'] = df.apply(lambda x: x.WFTA if x.id_diff <0 else x.LFTA, axis=1) 
    df['Team_a_OR'] = df.apply(lambda x: x.WOR if x.id_diff<0 else x.LOR, axis =1)
    df['Team_a_DR'] = df.apply(lambda x: x.WDR if x.id_diff<0 else x.LDR, axis =1)
    df['Team_a_Ast'] = df.apply(lambda x: x.WAst if x.id_diff<0 else x.LAst, axis =1)
    df['Team_a_TO'] = df.apply(lambda x: x.WTO if x.id_diff<0 else x.LTO, axis =1)
    df['Team_a_Stl'] = df.apply(lambda x: x.WStl if x.id_diff<0 else x.LStl, axis =1)
    df['Team_a_Blk'] = df.apply(lambda x: x.WBlk if x.id_diff<0 else x.LBlk, axis =1)
    df['Team_a_PF'] = df.apply(lambda x: x.WPF if x.id_diff<0 else x.LPF, axis =1)
    
    df['Team_b_Loc'] = df.apply(lambda x: x.LLoc if x.id_diff<0 else x.WLoc, axis=1)
    df['Team_b_FGM2_rate'] = df.apply(lambda x: (x.LFGM - x.LFGM3) / (x.LFGA - x.LFGA3) if x.id_diff <0 else (x.WFGM - x.WFGM3) / (x.WFGA - x.WFGA3), axis=1) 
    df['Team_b_FGM3_rate'] = df.apply(lambda x: x.LFGM3 / x.LFGA3 if x.id_diff <0 else x.WFGM3 / x.WFGA3, axis=1)
    df['Team_b_FTM_count'] = df.apply(lambda x: x.LFTA if x.id_diff <0 else x.WFTA, axis=1) 
    df['Team_b_OR'] = df.apply(lambda x: x.LOR if x.id_diff<0 else x.WOR, axis =1)
    df['Team_b_DR'] = df.apply(lambda x: x.LDR if x.id_diff<0 else x.WDR, axis =1)
    df['Team_b_Ast'] = df.apply(lambda x: x.LAst if x.id_diff<0 else x.WAst, axis =1)
    df['Team_b_TO'] = df.apply(lambda x: x.LTO if x.id_diff<0 else x.WTO, axis =1)
    df['Team_b_Stl'] = df.apply(lambda x: x.LStl if x.id_diff<0 else x.WStl, axis =1)
    df['Team_b_Blk'] = df.apply(lambda x: x.LBlk if x.id_diff<0 else x.WBlk, axis =1)
    df['Team_b_PF'] = df.apply(lambda x: x.LPF if x.id_diff<0 else x.WPF, axis =1)
    
    df['Score_diff'] = df.apply(lambda x: x.WScore - x.LScore if x.id_diff <0 else x.LScore-x.WScore, axis = 1) 
    for i in ['Season','WTeamID','LTeamID']:
        df[i] = df[i].apply(lambda x: str(x))
    df['ID'] = df.apply(lambda x: x.Season+'_'+x.WTeamID+'_'+x.LTeamID if x.id_diff <0 else x.Season +'_'+ x.LTeamID+'_'+x.WTeamID,axis= 1)
    
    return df

In [29]:
#Apply score_diff to both regular season and tourney
df_list = [df_regular_results,df_tourney_results]

df_sdiff = pd.DataFrame()
for i in df_list:
    temp = score_diff(i)
    print(temp.shape)
    df_sdiff = df_sdiff.append(temp)

print(df_sdiff.shape)
print(df_sdiff.head())

(87504, 62)
(1115, 62)
(88619, 62)
  Season  DayNum WTeamID  WScore LTeamID  LScore WLoc  NumOT  WFGM  WFGA  \
0   2003      10    1104      68    1328      62    N      0    27    58   
1   2003      10    1272      70    1393      63    N      0    26    62   
2   2003      11    1266      73    1437      61    N      0    24    58   
3   2003      11    1296      56    1457      50    N      0    18    38   
4   2003      11    1400      77    1208      71    N      0    30    61   

        ...        Team_b_FTM_count  Team_b_OR  Team_b_DR  Team_b_Ast  \
0       ...                      22         10         22           8   
1       ...                      20         20         25           7   
2       ...                      23         31         22           9   
3       ...                      15         17         20           9   
4       ...                      13         17         22          12   

   Team_b_TO  Team_b_Stl  Team_b_Blk  Team_b_PF  Score_diff          

In [33]:
#Check duplicates

dups = df_sdiff[df_sdiff.duplicated(['ID'],keep = False)]


dups = dups.sort_values('ID')

print(dups.shape)

print(dups.head())


(47871, 62)
     Season  DayNum WTeamID  WScore LTeamID  LScore WLoc  NumOT  WFGM  WFGA  \
4048   2003     117    1140      56    1102      43    A      0    19    34   
2946   2003      91    1140      65    1102      33    H      0    21    45   
3419   2003     103    1102      80    1161      75    H      0    23    32   
2201   2003      75    1161      51    1102      49    H      0    19    37   
1500   2003      58    1102      78    1204      40    H      0    27    40   

           ...        Team_b_FTM_count  Team_b_OR  Team_b_DR  Team_b_Ast  \
4048       ...                      22          5         24           6   
2946       ...                      23         17         38          16   
3419       ...                      10         11         13          12   
2201       ...                      20         11         23          12   
1500       ...                      16          9         11           6   

      Team_b_TO  Team_b_Stl  Team_b_Blk  Team_b_PF  Scor

In [38]:
dup_reg = df_regular_results[df_regular_results.duplicated(['Season','WTeamID','LTeamID'],keep = False)]

dup_reg.sort_values(['Season','WTeamID','LTeamID'],inplace = True)

print(dup_reg.shape)

print(dup_reg.head())

(30480, 35)
      Season  DayNum  WTeamID  WScore  LTeamID  LScore WLoc  NumOT  WFGM  \
763     2003      34     1102      70     1204      56    A      0    24   
1500    2003      58     1102      78     1204      40    H      0    27   
2020    2003      70     1105      66     1115      65    A      0    23   
3254    2003      98     1105      72     1115      54    H      0    22   
855     2003      40     1105      65     1289      58    A      0    20   

      WFGA  ...    LFTM  LFTA  LOR  LDR  LAst  LTO  LStl  LBlk  LPF  Label  
763     42  ...       7     8   14   17     7   15     6     1   20    Reg  
1500    40  ...       9    16    9   11     6   20    10     0   16    Reg  
2020    57  ...      17    21   14   26    13   21    11     5   22    Reg  
3254    58  ...      20    28   10   27    10   24    10     4   21    Reg  
855     64  ...      14    28   13   26     7   19     3     8   18    Reg  

[5 rows x 35 columns]


C:\Users\wongy\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [39]:
dup_tourney = df_tourney_results[df_tourney_results.duplicated(['Season','WTeamID','LTeamID'],keep = False)]

dup_tourney.sort_values(['Season','WTeamID','LTeamID'],inplace = True)

print(dup_tourney.shape)

print(dup_tourney.head())

(0, 35)
Empty DataFrame
Columns: [Season, DayNum, WTeamID, WScore, LTeamID, LScore, WLoc, NumOT, WFGM, WFGA, WFGM3, WFGA3, WFTM, WFTA, WOR, WDR, WAst, WTO, WStl, WBlk, WPF, LFGM, LFGA, LFGM3, LFGA3, LFTM, LFTA, LOR, LDR, LAst, LTO, LStl, LBlk, LPF, Label]
Index: []

[0 rows x 35 columns]


C:\Users\wongy\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [48]:
dup_check = dups[dups.Label == 'Tourney']

dup_check = dup_check.sort_values('ID')

print(dup_check.shape)

print(dup_check)



(52, 62)
     Season  DayNum WTeamID  WScore LTeamID  LScore WLoc  NumOT  WFGM  WFGA  \
57     2003     145    1242      78    1112      75    N      0    29    67   
127    2004     154    1163      82    1210      73    N      0    26    61   
112    2004     139    1462      89    1280      74    N      0    30    55   
182    2005     144    1277      78    1181      68    N      0    27    65   
247    2006     144    1206      63    1455      55    N      0    20    44   
250    2006     145    1417      50    1272      45    N      0    14    40   
243    2006     143    1400      74    1452      71    N      0    28    58   
319    2007     154    1196      84    1326      75    N      0    26    53   
310    2007     144    1207      66    1435      65    N      0    25    56   
307    2007     143    1326      85    1397      84    N      0    27    51   
301    2007     139    1356      63    1439      48    N      0    19    43   
438    2009     144    1277      67    1242

In [51]:
check = dups[dups.ID == '2003_1112_1242']

check

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,Team_b_FTM_count,Team_b_OR,Team_b_DR,Team_b_Ast,Team_b_TO,Team_b_Stl,Team_b_Blk,Team_b_PF,Score_diff,ID
2487,2003,82,1112,91,1242,74,A,0,30,57,...,12,11,17,19,17,12,6,21,17,2003_1112_1242
57,2003,145,1242,78,1112,75,N,0,29,67,...,18,15,22,16,15,13,5,20,-3,2003_1112_1242


## Thoughts about "duplicates"

## 1. A 对 B 对阵的次数 - New Label variable (maybe used in some way)
## 2. ID 保留，创建新的variable, ID_DayNum (unique), used to merge

In [52]:
#Keep only ID and Score_diff and merged with DataMart

df_sdiff_formerge = df_sdiff[[
                                'ID', 
                                'DayNum',
                                'Team_a_Loc',
                                'Team_a_FGM2_rate',
                                'Team_a_FGM3_rate',
                                'Team_a_FTM_count',
                                'Team_a_OR',
                                'Team_a_DR',
                                'Team_a_Ast',
                                'Team_a_TO',
                                'Team_a_Stl',
                                'Team_a_Blk',
                                'Team_a_PF',
                                'Team_b_Loc',
                                'Team_b_FGM2_rate',
                                'Team_b_FGM3_rate',
                                'Team_b_FTM_count',
                                'Team_b_OR',
                                'Team_b_DR',
                                'Team_b_Ast',
                                'Team_b_TO',
                                'Team_b_Stl',
                                'Team_b_Blk',
                                'Team_b_PF',
                                'Score_diff',
                                'Label',
                                'win'
                             ]]

df_datamart_add1 = pd.merge(left = df_datamart,right = df_sdiff_formerge, on = 'ID',how = 'left')

print(df_datamart_add1.shape)

print(df_datamart_add1.head())

(2376273, 30)
               ID  Season  TeamID_a  TeamID_b  DayNum Team_a_Loc  \
0  1985_1101_1102    1985      1101      1102     NaN        NaN   
1  1985_1101_1103    1985      1101      1103     NaN        NaN   
2  1985_1101_1104    1985      1101      1104     NaN        NaN   
3  1985_1101_1105    1985      1101      1105     NaN        NaN   
4  1985_1101_1106    1985      1101      1106     NaN        NaN   

   Team_a_FGM2_rate  Team_a_FGM3_rate  Team_a_FTM_count  Team_a_OR ...   \
0               NaN               NaN               NaN        NaN ...    
1               NaN               NaN               NaN        NaN ...    
2               NaN               NaN               NaN        NaN ...    
3               NaN               NaN               NaN        NaN ...    
4               NaN               NaN               NaN        NaN ...    

   Team_b_OR  Team_b_DR  Team_b_Ast  Team_b_TO  Team_b_Stl  Team_b_Blk  \
0        NaN        NaN         NaN        NaN      

In [17]:
check_missing(df_datamart_add1)

,col_names,col_dtypes,col_na_count,col_unique,count,mean,std,min,25%,50%,75%,max
ID,ID,object,0,335805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Season,Season,int64,0,5,343776.0,2017.000108,1.414250,2015.000000,2016.000000,2017.000000,2018.000000,2019.000000
TeamID_a,TeamID_a,int64,0,366,343776.0,1222.709927,86.379098,1101.000000,1150.000000,1208.000000,1284.000000,1466.000000
TeamID_b,TeamID_b,int64,0,366,343776.0,1345.352904,86.354654,1102.000000,1284.000000,1360.000000,1418.000000,1467.000000
Team_a_Loc,Team_a_Loc,object,316455,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Team_a_FGM2_rate,Team_a_FGM2_rate,float64,316455,607,27321.0,0.491186,0.096218,0.060606,0.428571,0.487805,0.555556,0.880000
Team_a_FGM3_rate,Team_a_FGM3_rate,float64,316455,327,27321.0,0.344757,0.111458,0.000000,0.272727,0.344828,0.416667,1.000000
Team_a_FTM_count,Team_a_FTM_count,float64,316455,57,27321.0,19.876066,7.795065,0.000000,14.000000,19.000000,25.000000,56.000000
Team_a_OR,Team_a_OR,float64,316455,33,27321.0,10.220234,3.979421,0.000000,7.000000,10.000000,13.000000,38.000000
Team_a_DR,Team_a_DR,float64,316455,44,27321.0,24.521174,5.168359,6.000000,21.000000,24.000000,28.000000,51.000000
